# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip",
                                     port="port번호",
                                     user="계정명",
                                     password="비밀번호",
                                     db="연결할데이터베이스이름",
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [13]:
create_sql = '''
create table customer(
id int auto_increment primary key,
name varchar(20) not null,
email varchar(50) not null unique,
tall decimal(5,2),
birthday date,
created_at datetime not null
)
'''

import pymysql

try:
    conn = None
    #1. Database와 연결
    conn = pymysql.connect(
                    host='127.0.0.1', #IP str
                    port=3306,       #port번호 int
                    user='playdata',  # str              
                    password='1111', # str
                    db = 'mydb' #str 
        )
    
    print(type(conn))
    
    #2.connection을 사용해서 cursor instance 생성
    # cursor : sql 처리를 하는 기능을 제공 (sql 전송, 처리 결과를 받을 때 까지를 관리)
    cursor = conn.cursor()
    
    
    #3. SQL 문 전송 + 처리결과 받기
    result = cursor.execute(create_sql)
    print(result)
finally:
    #4. 연결 닫기
    if conn:
        cursor.close() # 커서먼저
        conn.close()

<class 'pymysql.connections.Connection'>


OperationalError: (1050, "Table 'customer' already exists")

## DML

### insert


In [31]:
sql = '''insert into customer (name, email, tall, birthday, created_at) \
values(
'이순신',
'lee@naver.com',
185.23,
'2000-09-20',
curdate()
)'''


# manual commit이 default. DML 커밋 필요
# try

with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print(result)
        conn.commit()


1


### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   query문을 쉽게 작성할 수 있는 장점이 있다.


In [89]:
insert_sql = """insert into customer (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, now())"""

# with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
#     with conn.cursor() as cursor:
#         result = cursor.execute(insert_sql, ['유관순', 'a', 213.00, '2011-02-04'])
#         print(result)
#         conn.commit()


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용


In [93]:
from datetime import date, datetime

datas = [
    ["name1", "ab2c1@abc.com", 165, date(2000, 1, 12), datetime.now()],
    ["name2", "de2f1@abc.com", 175, date(1995, 12, 20), datetime.now()],
    ["name3", "gh2i1@abc.com", 185, date(1988, 7, 21), datetime.now()],
]
insert_sql = """insert into customer (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, %s)"""

with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        for data in datas:
            result = cursor.execute(insert_sql, data)
            print(result)
            conn.commit()

1
1
1


#### executemany() 사용

-   insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [95]:
from datetime import datetime, date

datas = [
    ["이름1", "ab2c22@a.com", 165, date(2000, 1, 12), datetime.now()],
    ["이름2", "de22f2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["이름1", "ghi222@a.com", 185, date(1988, 7, 21), datetime.now()],
]
insert_sql = "insert into customer (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, %s)"


with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.executemany(insert_sql, datas)
        print(result)
        conn.commit()


3


### update/delete

-   코딩 절차는 insert 와 동일


In [99]:
update_sql = """update customer set tall=%s where id =%s"""
f = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [f, cust_id])
        print(result)
        conn.commit()

변경할 키:  190
변경할 고객 ID:  3


0


In [100]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준키: "))

with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, tall)
        print(result)
        conn.commit()

삭제기준키:  179


7


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   지정한 size개수 만큼 반환
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [104]:
sql = 'select * from customer'


with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        conn.commit()
        t = cursor.fetchall()
        for c in t:
            print(c)



(12, 'name1', 'abc1@abc.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 7, 41))
(13, 'name2', 'def1@abc.com', Decimal('175.00'), datetime.date(1995, 12, 20), datetime.datetime(2024, 3, 22, 16, 7, 41))
(15, '이름1', 'abc2@a.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 9))
(16, '이름2', 'def2@a.com', Decimal('175.00'), datetime.date(1995, 12, 20), datetime.datetime(2024, 3, 22, 16, 9))
(22, 'name1', 'ab2c1@abc.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 11, 15))
(23, 'name2', 'de2f1@abc.com', Decimal('175.00'), datetime.date(1995, 12, 20), datetime.datetime(2024, 3, 22, 16, 11, 15))
(25, '이름1', 'abc22@a.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 11, 21))
(26, '이름2', 'de2f2@a.com', Decimal('175.00'), datetime.date(1995, 12, 20), datetime.datetime(2024, 3, 22, 16, 11, 21))
(28, '이름1', 'ab2c22@a.com', Decimal('165.00'), datet

In [107]:
with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb', cursorclass=pymysql.cursors.DictCursor) as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        conn.commit()
        t = cursor.fetchall()
        for c in t:
            print(c)


{'id': 12, 'name': 'name1', 'email': 'abc1@abc.com', 'tall': Decimal('165.00'), 'birthday': datetime.date(2000, 1, 12), 'created_at': datetime.datetime(2024, 3, 22, 16, 7, 41)}
{'id': 13, 'name': 'name2', 'email': 'def1@abc.com', 'tall': Decimal('175.00'), 'birthday': datetime.date(1995, 12, 20), 'created_at': datetime.datetime(2024, 3, 22, 16, 7, 41)}
{'id': 15, 'name': '이름1', 'email': 'abc2@a.com', 'tall': Decimal('165.00'), 'birthday': datetime.date(2000, 1, 12), 'created_at': datetime.datetime(2024, 3, 22, 16, 9)}
{'id': 16, 'name': '이름2', 'email': 'def2@a.com', 'tall': Decimal('175.00'), 'birthday': datetime.date(1995, 12, 20), 'created_at': datetime.datetime(2024, 3, 22, 16, 9)}
{'id': 22, 'name': 'name1', 'email': 'ab2c1@abc.com', 'tall': Decimal('165.00'), 'birthday': datetime.date(2000, 1, 12), 'created_at': datetime.datetime(2024, 3, 22, 16, 11, 15)}
{'id': 23, 'name': 'name2', 'email': 'de2f1@abc.com', 'tall': Decimal('175.00'), 'birthday': datetime.date(1995, 12, 20), 'crea

### fetchone()


### fetchmany()


### cursor 는 iterable 타입

-   for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


# TODO

다음 함수들을 구현하시오.  
각 함수들은 member 테이블과 관련해서 CRUD를 처리하는 함수들 입니다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)
2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)
3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.
4. 이름을 매개변수로 받아서 그 이름의 member를 삭제하는 함수.
5. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.
6. name을 매개변수로 받아서 그 이름이 들어간 회원의 정보를 조회하여 반환하는 함수.
7. birthday를 매개변수로 받아서 그 생일의 회원의 정보를 조회하여 반환하는 함수.
8. tall 값을 두개 받아서 그 범위의 tall인 회원들의 정보를 조회하여 반환하는 함수.

-   insert, update, delete 는 적용된 행의 개수를 반환한다.
-   select 처리 함수는 조회결과를 반환한다.


In [126]:
def cursor_connect(*args):
    print(args)
    with pymysql.connect(host='127.0.0.1', port=3306, user='playdata', password='1111', db = 'mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(*args)
            conn.commit()
            print("결과: ", result)
            if 'select' in args[0]: 
                t = cursor.fetchall()
                for c in t:
                    print(c)

def parameters(*args):
    rl = []
    for a in args:
        rl.append(input(a))
    return rl

In [ ]:
sql_1 = """insert into customer (name, email, tall, birthday, created_at) values (%s,%s,%s,%s,%s)"""
t = parameters('name','email','tall','birthday')
t.append(datetime.now())
cursor_connect(sql_1,t)

In [ ]:
id_num=input('수정할 아이디')
t = parameters('name','email','tall','birthday')
sql_2 = """update customer set name=%s, email=%s, tall=%s, birthday=%s where id="""+id_num
cursor_connect(sql_2,t)

In [ ]:
id_num=input('삭제할 아이디') 
sql_3 = 'delete from customer where id='+id_num
cursor_connect(sql_3)

In [156]:
id_num=input('조회할 아이디') 
sql_4 = 'select * from customer where id='+id_num
cursor_connect(sql_4)

조회할 아이디 12


('select * from customer where id=12',)
결과:  1
(12, 'name1', 'abc1@abc.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 7, 41))


# TODO

-   hr database의 emp 테이블의 데이터를 다루는 함수들을 구현

1. 직원 정보들(emp의 컬럼들)을 매개변수로 받아 insert 하는 함수

2. 수정할 직원 정보들을 매개변수로 받아 받은 emp_id의 직원의 나머지 컬럼값들을 수정하는 함수

3. emp_id를 매개변수로 받아 그 직원을 삭제하는 함수

4. job을 매개변수로 받아 그 업무를 하는 직원들을 삭제하는 함수

5. 전체 직원정보를 모두 조회해서 반환하는 함수

6. emp_id를 매개변수로 받아 그 직원 정보를 조회해서 반환하는 함수

7. dept_name을 매개변수로 받아서 그 부서에 소속된 직원들의 정보를 조회해서 반환하는 함수

8. salary 범위를 매개변수로 받아 그 범위의 salary를 받는 직원들의 정보를 조회해서 반환하는 함수

9. emp_id를 매개변수로 받아서 그 직원의 커미션이 얼마인지를(salary \* comm_pct) 조회해 반환하는 함수

10. dept_name을 매개변수로 받아서 부서별 급여 통계정보(합계, 최대, 최소, 평균, 표준편차)를 반환하는 함수
